## Understanding LSTM

**Notes:** https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

**Video:** https://youtu.be/8HyCNIVRbSU

### Example

https://www.geeksforgeeks.org/next-word-prediction-with-deep-learning-in-nlp/

In [55]:
!pip install tensorflow

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [23]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
 
    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(r'(?<=[.!?])\s+', text) if sentence.strip()]
 
    return sentences

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

text_data

['Pizza, the delectable and iconic dish that has transcended borders and captivated taste buds worldwide, is a testament to the extraordinary fusion of flavors, creativity, and cultural significance.',
 'Originating from the sun-kissed lands of Italy, pizza has evolved into an art form that unites people from diverse backgrounds in a shared love for its mouthwatering combinations.',
 'Its history stretches back centuries, with roots tracing back to ancient civilizations like the Greeks, Romans, and Egyptians, who all had their versions of flatbreads adorned with various ingredients.',
 'However, it was the vibrant city of Naples, Italy, that birthed the pizza we know and adore today.',
 'With its soft and chewy Neapolitan crust, topped with the perfect balance of tomatoes, mozzarella cheese, and fresh basil, the Margherita pizza pays homage to Queen Margherita of Italy and embodies the colors of the Italian flag.',
 'As pizza migrated from the shores of Naples, it found its way to the 

In [24]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

total_words

687

In [25]:
# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

[[3, 1],
 [3, 1, 233],
 [3, 1, 233, 2],
 [3, 1, 233, 2, 45],
 [3, 1, 233, 2, 45, 35],
 [3, 1, 233, 2, 45, 35, 13],
 [3, 1, 233, 2, 45, 35, 13, 8],
 [3, 1, 233, 2, 45, 35, 13, 8, 114],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14, 5],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14, 5, 116],
 [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14, 5, 116, 6],
 [3,
  1,
  233,
  2,
  45,
  35,
  13,
  8,
  114,
  234,
  2,
  235,
  74,
  115,
  236,
  14,
  5,
  116,
  6,
  1],
 [3,
  1,
  233,
  2,
  45,
  35,
  13,
  8,
  114,
  234,
  2,
  235,
  74,
  115,
  23

In [26]:
# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences


array([[  0,   0,   0, ...,   0,   3,   1],
       [  0,   0,   0, ...,   3,   1, 233],
       [  0,   0,   0, ...,   1, 233,   2],
       ...,
       [  0,   0,   0, ..., 685,   4,  19],
       [  0,   0,   0, ...,   4,  19,  72],
       [  0,   0,   0, ...,  19,  72, 686]])

In [27]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [28]:
X

array([[  0,   0,   0, ...,   0,   0,   3],
       [  0,   0,   0, ...,   0,   3,   1],
       [  0,   0,   0, ...,   3,   1, 233],
       ...,
       [  0,   0,   0, ..., 684, 685,   4],
       [  0,   0,   0, ..., 685,   4,  19],
       [  0,   0,   0, ...,   4,  19,  72]])

In [29]:
y

array([  1, 233,   2, ...,  19,  72, 686])

In [30]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [31]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

- In Keras, the Sequential model is a linear stack of layers. You can add one layer at a time using this model.
- Embedding layer is used for word embedding. It turns positive integers (indexes) into dense vectors of fixed size.
- total_words is the size of the vocabulary (number of unique words in your dataset).
- 10 is the dimensionality of the embedding vector. Each word will be represented by a vector of length 10.
- input_length=max_sequence_len-1 indicates the length of input sequences. It's likely that your model is trained on sequences of words, and max_sequence_len is the maximum length of these sequences.
- Adds a Long Short-Term Memory (LSTM) layer with 128 units
- Adds a densely connected layer with total_words units (one for each unique word in your vocabulary).
- softmax activation function is used to convert the output to probabilities. It's often used in multi-class classification problems.
- categorical_crossentropy is the loss function used, suitable for multi-class classification problems.
- adam is an optimization algorithm.
- accuracy is used as a metric to monitor during training.

In [34]:
# Train the model
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
51/51 [==============================] - 2s 31ms/step - loss: 5.3363 - accuracy: 0.0774
Epoch 2/100
51/51 [==============================] - 2s 31ms/step - loss: 5.2325 - accuracy: 0.0854
Epoch 3/100
51/51 [==============================] - 3s 52ms/step - loss: 5.1232 - accuracy: 0.0952
Epoch 4/100
51/51 [==============================] - 2s 34ms/step - loss: 4.9930 - accuracy: 0.1057
Epoch 5/100
51/51 [==============================] - 3s 52ms/step - loss: 4.8694 - accuracy: 0.1050
Epoch 6/100
51/51 [==============================] - 2s 42ms/step - loss: 4.7425 - accuracy: 0.1106
Epoch 7/100
51/51 [==============================] - 2s 41ms/step - loss: 4.6268 - accuracy: 0.1192
Epoch 8/100
51/51 [==============================] - 2s 40ms/step - loss: 4.5119 - accuracy: 0.1259
Epoch 9/100
51/51 [==============================] - 2s 39ms/step - loss: 4.4010 - accuracy: 0.1314
Epoch 10/100
51/51 [==============================] - 2s 33ms/step - loss: 4.2993 - accuracy: 0.1351

51/51 [==============================] - 2s 32ms/step - loss: 0.4887 - accuracy: 0.9380
Epoch 83/100
51/51 [==============================] - 2s 37ms/step - loss: 0.4764 - accuracy: 0.9380
Epoch 84/100
51/51 [==============================] - 2s 33ms/step - loss: 0.4636 - accuracy: 0.9386
Epoch 85/100
51/51 [==============================] - 2s 34ms/step - loss: 0.4524 - accuracy: 0.9398
Epoch 86/100
51/51 [==============================] - 2s 33ms/step - loss: 0.4405 - accuracy: 0.9410
Epoch 87/100
51/51 [==============================] - 1s 29ms/step - loss: 0.4299 - accuracy: 0.9423
Epoch 88/100
51/51 [==============================] - 2s 31ms/step - loss: 0.4172 - accuracy: 0.9404
Epoch 89/100
51/51 [==============================] - 2s 31ms/step - loss: 0.4067 - accuracy: 0.9466
Epoch 90/100
51/51 [==============================] - 2s 30ms/step - loss: 0.3949 - accuracy: 0.9435
Epoch 91/100
51/51 [==============================] - 2s 30ms/step - loss: 0.3867 - accuracy: 0.9453
Epo

In [36]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5
 
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word
 
print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 29ms/step
Next predicted words: Pizza have different  become a symbol of comfort
